In [1]:
using Dates, DataFrames, CSV, JSON, Plots, JuMP, HiGHS, TuLiPa

[ Info: Precompiling TuLiPa [970f5c25-cd7d-4f04-b50d-7a4fe2af6639]


In [2]:
firm_demand = 50.0
price_elasticity = -0.05
normal_price = 100.0
max_price = 150.0
min_price = 50.0
CapNucGER = 5000.0
GenNucGER = 100.0


elements = [
    DataElement("Balance", "BaseBalance", "PowerBalance_NO2", Dict{Any, Any}("Commodity" => "Power")),    
    DataElement("Flow", "BaseFlow", "GenNucGER", Dict{Any, Any}()),
    DataElement("Arrow", "BaseArrow", "PowerBalance_NO2", Dict{Any, Any}("Balance" => "PowerBalance_NO2", "Flow" => "GenNucGER", "Conversion" => 1.0, "Direction" => "In")),
    DataElement("Cost", "CostTerm", "MCNucGER", Dict{Any, Any}("Param" => GenNucGER, "WhichInstance" => "GenNucGER", "WhichConcept" => "Flow", "Direction" => "In")),
    DataElement("Capacity", "PositiveCapacity", "CapNucGER", Dict{Any, Any}("Param" => "CapNucGER", "WhichInstance" => "GenNucGER", "WhichConcept" => "Flow", "Bound" => "Upper")),
    DataElement("Param", "MWToGWhSeriesParam", "CapNucGER", Dict{Any, Any}("Level" => CapNucGER, "Profile" => 1.0)),

    DataElement("Param", "MeanSeriesParam", "FirmDemand", Dict{Any, Any}("Level" => firm_demand, "Profile" => 1.0)),
    
    DataElement("ElasticDemand", "BaseElasticDemand", "", 
                Dict{Any, Any}("Balance" => "PowerBalance_NO2", "Param" => "FirmDemand",
                "price_elasticity" => price_elasticity, "normal_price" => normal_price , "max_price" => max_price, "min_price" => min_price
    ))   
        
]

power_horizon = SequentialHorizon(1, Day(1))
push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", "Power", (HORIZON_CONCEPT, power_horizon)))
addscenariotimeperiod!(elements, "ScenarioTimePeriod", getisoyearstart(1981), getisoyearstart(1983));

In [3]:
modelobjects = getmodelobjects(elements)

Dict{Id, Any} with 3 entries:
  Id("ElasticDemand", "")           => BaseElasticDemand(Id("ElasticDemand", ""…
  Id("Flow", "GenNucGER")           => BaseFlow(Id("Flow", "GenNucGER"), Sequen…
  Id("Balance", "PowerBalance_NO2") => BaseBalance(Id("Balance", "PowerBalance_…

In [4]:
mymodel = JuMP.Model(HiGHS.Optimizer)
prob = JuMP_Prob(modelobjects, mymodel)

JuMP_Prob(A JuMP Model
Minimization problem with:
Variables: 11
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: BalancePowerBalance_NO2, ElasticDemand1, ElasticDemand10, ElasticDemand2, ElasticDemand3, ElasticDemand4, ElasticDemand5, ElasticDemand6, ElasticDemand7, ElasticDemand8, ElasticDemand9, FlowGenNucGER, Any[BaseElasticDemand(Id("ElasticDemand", ""), BaseBalance(Id("Balance", "PowerBalance_NO2"), BaseCommodity(Id("Commodity", "Power"), SequentialHorizon(SequentialPeriods([(1, Millisecond(86400000))]), nothing)), SequentialHorizon(SequentialPeriods([(1, Millisecond(86400000))]), nothing), RHSTerm[], Dict{Any, Any}()), MeanSeriesParam{ConstantTimeVector, ConstantTimeVector}(ConstantTimeV

In [5]:
for (i, v) in enumerate(prob.objects)
    if v.id == Id("ElasticDemand", "ElasticDemand")
        println("segment_capacities ", v.segment_capacities)
        println("reserve_prices ", v.reserve_prices)
    end
end

In [6]:
# Price increase by 1% from normal price, then demand decrease 1%, with ealsticity = -1
p1 = price_to_relative_demand(100, -1.0, 101)

# Price increase by 1% from normal price, then demand decrease 0.05%, with ealsticity = -0.05
p2 = price_to_relative_demand(100, -0.05, 101)

p1, p2

(0.9900990099009901, 0.9995026071981749)

In [7]:
datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TwoTime(datatime, scenariotime)
update!(prob, prob_time)
solve!(prob)

In [8]:
print(prob.model)

In [9]:
solution_summary(prob.model, verbose = true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -2.46598e+03
  Objective bound    : -2.46598e+03
  Relative gap       : Inf
  Dual objective value : -2.46598e+03
  Primal solution :
    ElasticDemand10[1] : 0.00000e+00
    ElasticDemand1[1] : 4.89965e+01
    ElasticDemand2[1] : 3.07411e-01
    ElasticDemand3[1] : 3.07411e-01
    ElasticDemand4[1] : 3.07411e-01
    ElasticDemand5[1] : 0.00000e+00
    ElasticDemand6[1] : 0.00000e+00
    ElasticDemand7[1] : 0.00000e+00
    ElasticDemand8[1] : 0.00000e+00
    ElasticDemand9[1] : 0.00000e+00
    FlowGenNucGER[1] : 4.99188e+01
  Dual solution :
    BalancePowerBalance_NO2[1] : -1.00000e+02

* Work counters
  Solve time (sec)   : 4.97818e-04
  Simplex iterations : 0
  Barrier iterations : 0
  Node count         : -1


In [10]:
ed = modelobjects[Id("ElasticDemand", "")]
getdemand(prob, ed, 1)

49.918777574441556